In [ ]:
## imports
import tensorflow as tf
from fastai.vision.all import *
import torch
import time

## TensorFlow Approach

In [ ]:
# Loading in the CIFAR-10 dataset (found it to be much more convienent to use a tf dataset)
(x_train, _), (_, _) = tf.keras.datasets.cifar10.load_data()

# Load the pre-trained ResNet50 model
model = tf.keras.applications.ResNet50(weights="imagenet")

# number of images to perform inference on
num_images = 10000

# Create a list to store the inference times
inference_times = []

In [ ]:
# Perform inference on the images and measure the inference time
for i in range(num_images):
    # Preprocess the image
    img = tf.image.resize(x_train[i], (224, 224))
    img = tf.expand_dims(img, axis=0)

    # Perform inference and measure the time
    start_time = time.time()
    preds = model.predict(img)
    end_time = time.time()

    # Calculate the inference time in milliseconds
    inference_time = (end_time - start_time) * 1000

    # Add the inference time to the list
    inference_times.append(inference_time)

In [ ]:
# Calculate the average/total inference time
average_inference_time = sum(inference_times) / len(inference_times)
total_inference_time = sum(inference_times)
print(f"\nAverage Inference Time: {average_inference_time:.2f} ms")
print(f"\nTotal Inference Time: {total_inference_time:.2f} ms")

## Fastai/Pytorch approach

In [ ]:
# Set the path to the Dogs vs. Cats dataset
# This is a 37 category pet dataset with roughly 200 images for each class, about 7400 images in total
path = untar_data(URLs.PETS)/"images"

# Created a DataBlock for the dataset, I used a 80/20 split for the data. Also used data augmentation.
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   splitter=RandomSplitter(valid_pct=0.2),
                   get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
# Create a DataLoaders object for the dataset
dls = dblock.dataloaders(path, bs=64)

In [ ]:
# Shows us an example of what our data looks like
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# Load the pre-trained ResNet model
learn = cnn_learner(dls, resnet34, pretrained=True, metrics=accuracy)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
## fine tuning here is basically the process of using resnet18's pre-trained previous weights for all layers
## but the last and we train the last layer using the pets data. We then use our new learning rate to imrpove the weights of all of the 
## other layers
learn.fine_tune(4)

In [ ]:
# Create a list to store the inference times
inference_times = []

for batch in dls.train + dls.valid:
    # Get the batch of images
    images = batch[0]

    start_time = time.time()
    preds = learn.get_preds(dl=batch, with_decoded=True)
    end_time = time.time()


    # Calculate the inference time in milliseconds
    inference_time = (end_time - start_time) * 1000

    # Add the inference time to the list
    inference_times.append(inference_time)

# Calculate the average/total inference time
average_inference_time = sum(inference_times) / len(inference_times)
total_inference_time = sum(inference_times)
print(f"\nAverage Inference Time per batch of 64 images: {average_inference_time:.2f} ms")
print(f"\nTotal Inference Time: {total_inference_time:.2f} ms")